In [22]:
import numpy as np
import random
import plotly.plotly as py
import plotly.graph_objs as go

In [23]:
def get_random_coordinates(limit_x, limit_y, num_limit):
    """
        Method which returns a set of random coordinates. 
        @Arguments: 
            limit_x: The upper bound on x coordinate
            limit_y: The upper bound on y coordinate
            num_limit: The upper bound on number of samples.
        @returns:
            coordinates: A list of 20 coordinates. 
    """
    coordinates = set()
    
    while(len(coordinates)<=num_limit):
        x = random.uniform(1, limit_x)
        y = random.uniform(1, limit_y)
        
        coordinates.add((x,y))
        
    return coordinates

In [24]:
dataset = get_random_coordinates(10, 10, 19)

x_coords = []
y_coords = []

# Considering everything on the left of the decision line as a negative example. 
neg_examples_nobias = []
pos_examples_nobias = []

for i in dataset: 
    x_coords.append(i[0])
    y_coords.append(i[1])
    if i[0] <= 5: 
        neg_examples_nobias.append(np.array(i))
    else: 
        pos_examples_nobias.append(np.array(i))
    
trace = go.Scatter(
    x = x_coords,
    y = y_coords,
    mode = 'markers', 
    name = 'data points'
)

line_trace = go.Scatter(
    x = [5,5],
    y = [0,10],
    mode = 'lines',
    name = 'decision curve'
)

data = [trace, line_trace]
py.iplot(data, filename='basic-data')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kkgoswami/0 or inside your plot.ly account where it is named 'basic-data'


In [26]:
import pickle 
with open('dataset.pickle','w') as f:
    pickle.dump(dataset, f)

In [42]:
neg_examples_nobias = np.vstack(neg_examples_nobias)
pos_examples_nobias = np.vstack(pos_examples_nobias)
[neg_examples, pos_examples, neg_example_mistakes, pos_example_mistakes, w, numiter] = learn_perceptron(np.array(neg_examples_nobias),np.array(pos_examples_nobias))
plot_perceptron(neg_examples, pos_examples, neg_example_mistakes, pos_example_mistakes, w, numiter)

Number of errors : 9
Weights: 
[ 1.85173307  4.94460504  0.29104408]
Number of errors : 9
Weights: 
[ 2.96966425  9.80676465  0.29104408]
Number of errors : 9
Weights: 
[ 2.423588    9.54005344 -0.70895592]
Number of errors : 9
Weights: 
[ 1.87751175  9.27334222 -1.70895592]
Number of errors : 9
Weights: 
[ 1.3314355   9.00663101 -2.70895592]
Number of errors : 9
Weights: 
[ 0.78535925  8.7399198  -3.70895592]
Number of errors : 9
Weights: 
[ 0.239283    8.47320859 -4.70895592]
Number of errors : 9
Weights: 
[-0.30679325  8.20649738 -5.70895592]
Number of errors : 9
Weights: 
[ 7.28018758  8.94168943 -5.70895592]
Number of errors : 9
Weights: 
[ 6.73411133  8.67497822 -6.70895592]
Number of errors : 9
Weights: 
[ 6.18803508  8.40826701 -7.70895592]
Number of errors : 9
Weights: 
[ 5.64195883  8.1415558  -8.70895592]
Number of errors : 9
Weights: 
[ 5.09588258  7.87484459 -9.70895592]
Number of errors : 9
Weights: 
[  4.54980633   7.60813337 -10.70895592]
Number of errors : 9
Weights: 


In [27]:
def learn_perceptron(neg_examples_nobias,pos_examples_nobias):
    """
        This function monitors the learning process of the perceptron. 
    """
    num_neg_examples = (neg_examples_nobias).shape
    num_pos_examples = (pos_examples_nobias).shape
    num_errors_history = []
        
    #Adding a column of ones to learn the bias parameters. 
    neg_examples = np.ones([num_neg_examples[0], num_neg_examples[1]+1])
    neg_examples[:,:-1] = neg_examples_nobias
    pos_examples = np.ones([num_pos_examples[0], num_pos_examples[1]+1])
    pos_examples[:,:-1] = pos_examples_nobias
        
    w = np.array([np.random.random() for i in [1,2,3]])
    
    
    w_gen_feas = np.zeros(shape=w.shape)
    
    
    
    neg_example_mistakes, pos_example_mistakes = eval_perceptron(neg_examples, pos_examples, w) 
    
    total_errors = len(neg_example_mistakes) + len(pos_example_mistakes)
    num_errors_history.append(total_errors)
    numiter = 0
    while total_errors > 0:
        
        w = update_weights(neg_examples, pos_examples, w)
        neg_example_mistakes, pos_example_mistakes = eval_perceptron(neg_examples, pos_examples, w) 
        total_errors = len(neg_example_mistakes) + len(pos_example_mistakes)
        num_errors_history.append(total_errors)
        #plot_perceptron(neg_examples, pos_examples, neg_example_mistakes, pos_example_mistakes, w, numiter)
        print "Number of errors : {}".format(total_errors)
        print "Weights: "
        print w 
        numiter += 1
        
        #key = raw_input('Press enter to continue, q to quit:')
        #if key is 'q':
        #    break
    print "Number of iterations: {}".format(numiter)
    return neg_examples, pos_examples, neg_example_mistakes, pos_example_mistakes, w, numiter

In [28]:
def update_weights(neg_examples, pos_examples, w_current): 
    """
        This function learns the weights for the perceptron. It 
        tries to improve the approximation of the curve which 
        can model the data perfectly. 
    """
    w = w_current
    
    for i in xrange(0, len(neg_examples)):
        this_case = neg_examples[i][:]
        x = (np.array(this_case)).transpose()
        
        activation = np.sum(np.array(this_case) * np.array(w))
        if activation >= 0: 
            w = (np.array(w) - np.array(x))
    
    for i in xrange(0, len(pos_examples)):
        this_case = pos_examples[i][:]
        x = (np.array(this_case)).transpose()
        
        activation = np.sum(np.array(this_case) * np.array(w))
        if activation < 0: 
            w = (np.array(w) + np.array(x))
            
    return w

In [29]:
def eval_perceptron(neg_examples, pos_examples, w): 
    """
        This function evaluates the performance of the model. 
        Determines the number of examples it classifies 
        correctly, and the number of incorrectly classified
        examples. 
    """
    neg_example_mistakes = [];
    pos_example_mistakes = [];
    
    for i in xrange(0,len(neg_examples)): 
        x = (neg_examples[i][:]).transpose()
        activation = np.sum((np.array(x)).transpose() * np.array(w))
        if activation >= 0:
            neg_example_mistakes.append(i)
            
    for i in xrange(0,len(pos_examples)): 
        x = (pos_examples[i][:]).transpose()
        activation = np.sum((np.array(x)).transpose() * np.array(w))
        if activation < 0:
            pos_example_mistakes.append(i)
            
    return neg_example_mistakes, pos_example_mistakes

In [41]:
def plot_perceptron(neg_examples, pos_examples, neg_example_mistakes, pos_example_mistakes, w, numiters):
    """
        Function to plot the graph of the original function versus the derived hypothesis. 
    """
    #print neg_example_mistakes
    #print pos_example_mistakes
    
    green_dots = []
    red_dots = []
    
    
    
    for i in xrange(0, len(neg_examples)):
        if i in neg_example_mistakes:
            red_dots.append(neg_examples[i])
        else:
            green_dots.append(neg_examples[i])

    for i in xrange(0, len(pos_examples)):
        if i in pos_example_mistakes:
            red_dots.append(pos_examples[i])
        else:
            green_dots.append(pos_examples[i])
            
    green_x = []
    green_y = []
    red_x = []
    red_y = []
    
    for i in green_dots:
        green_x.append(i[0])
        green_y.append(i[1])
        
    for i in red_dots:
        red_x.append(i[0])
        red_y.append(i[1])
        
    
    approx_curve_slope, approx_curve_bias = -w[0]/w[1], -w[1]/w[2]
    
    approx_y = np.linspace(0, 10)
    approx_x = ((approx_y - approx_curve_bias)/(approx_curve_slope)) + 6.5
                
    green_trace = go.Scatter(
        x = green_x,
        y = green_y,
        mode = 'markers', 
        name = 'Correctly classified points'
    )
    
    red_trace = go.Scatter(
        x = red_x,
        y = red_y,
        mode = 'markers', 
        name = 'Incorrectly classified points'
    )
    
    original_trace = go.Scatter(
        x = [5,5],
        y = [0,10],
        mode = 'lines',
        name = 'decision curve'
    )
    
    approx_trace = go.Scatter(
        x = approx_x, 
        y = approx_y,
        mode = 'lines', 
        name = 'Approximated curve'
    )
    

    data = [green_trace, red_trace, original_trace, approx_trace]
    layout = go.Layout(title='Evaluating Iteration: {}'.format(numiters), width=800, height=640)
    fig = go.Figure(data=data, layout=layout)

    py.image.save_as(fig, filename='eval{}.png'.format(numiters))
    #py.iplot(data, filename='eval_data{}'.format(numiters))
        
    

In [20]:
print np.sign(-0.5)

-1.0
